In [1]:
import os
import sys
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm
from warnings import warn
%matplotlib inline

C:\Users\x_j_t\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def pgm(filePath, byteorder='>'):
    with open(filePath,'rb') as f:
        file=f.read()
    try:
        header, width, height, maxval=re.search(b"(^P5\s(?:\s*#.*[\r\n])*"
                                                b"(\d+)\s(?:\s*#.*[\r\n])*"
                                                b"(\d+)\s(?:\s*#.*[\r\n])*"
                                                b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", file).groups()
    except AttributeError:
        raise ValueError("Not a PGM file: {}".format(filePath))
    return np.frombuffer(file, dtype='u1' if int(maxval)<256 else byteorder+'u2',
                         count=int(width)*int(height),
                         offset=len(header)).reshape((int(height), int(width)))

In [7]:
data_df=pd.read_csv('../DATA/infoTable.txt', delimiter=' ', error_bad_lines=False)
#data_df=data_df[data_df.columns[:-1]]
data_df['PATH']=data_df['PATH'].map(lambda x:'../DATA/%s.pgm'%x)
data_df = data_df[pd.notnull(data_df['SEVERITY'])]
data_df.head()

,PATH,BG,CLASS,SEVERITY,X,Y,RADIUS
0,../DATA/mdb001.pgm,G,CIRC,B,535,425,197.0
1,../DATA/mdb002.pgm,G,CIRC,B,522,280,69.0
4,../DATA/mdb005.pgm,F,CIRC,B,477,133,30.0
5,../DATA/mdb005.pgm,F,CIRC,B,500,168,26.0
10,../DATA/mdb010.pgm,F,CIRC,B,525,425,33.0


In [8]:
data_df['scan']=data_df['PATH'].map(pgm)

In [9]:
def write_h5(out_path, out_tf):
    with h5py.File(out_path,'w') as f:
        for k, arr_dict in tqdm(out_tf.to_dict().items()):
            try: 
                s_data=np.stack(arr_dict.values(), 0)
                try:
                    f.create_dataset(k, data=s_data, compression='gzip')
                except TypeError as e:
                    try:
                        f.create_dataset(k, data=s_data.astype(np.string_))
                    except TypeError as e2:
                        print("%s cannot be added to h5 file, error code: %s\n"%(k, repr(e)))
            except ValueError as e:
                print("%s could not be cerated, %s"%(k,repr(e)))
                all_shape=[np.shape(x) for x in arr_dict.values()]
write_h5('mias.h5', data_df)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.81it/s]
